In [1]:
import torch
import random

from utils import corrupt_word, edit_distance, encode_word, decode_indices
from models import Encoder, Decoder, Seq2Seq

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

model_path = 'georgian_spellcheck_seq2seq.pt'
checkpoint = torch.load(model_path, map_location = device)

itos = checkpoint['vocab']['itos']
stoi = checkpoint['vocab']['stoi']
pad_idx = checkpoint['vocab']['pad_idx']
sos_idx = checkpoint['vocab']['sos_idx']
eos_idx = checkpoint['vocab']['eos_idx']
unk_idx = checkpoint['vocab']['unk_idx']

embedding_dim = checkpoint['config']['embedding_dim']
hidden_dim = checkpoint['config']['hidden_dim']
num_layers = checkpoint['config']['num_layers']
dropout = checkpoint['config']['dropout']
max_target_len = checkpoint['config']['max_target_len']

vocab_size = len(itos)

Using device: cpu


In [3]:
encoder = Encoder(vocab_size, embedding_dim, hidden_dim, pad_idx, num_layers, dropout)
decoder = Decoder(vocab_size, embedding_dim, hidden_dim, pad_idx, num_layers, dropout)

model = Seq2Seq(
    encoder,
    decoder,
    sos_idx,
    eos_idx,
    pad_idx,
    max_target_len,
    vocab_size
).to(device)

model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print('Model loaded')

Model loaded


In [4]:
def correct_word(word, model = model):
    src_indices = encode_word(word, stoi, unk_idx, eos_idx)
    src_tensor = torch.tensor(src_indices, dtype = torch.long).unsqueeze(0).to(device)
    src_length = torch.tensor([len(src_indices)], dtype = torch.long).to(device)

    with torch.no_grad():
        logits = model(src_tensor, src_length, teacher_forcing_ratio = 0.0)

    token_indices = logits.argmax(dim = 2)[0].tolist()
    return decode_indices(token_indices, itos, eos_idx, pad_idx)

In [5]:
with open('georgian_words.txt', 'r', encoding = 'utf-8') as f:
    all_words_list = [w for w in f.read().split(',') if w]

print("Loaded words:", len(all_words_list))

Loaded words: 23162


In [6]:
random.seed(42)

sample_words = random.sample(all_words_list, 1000)

results = []

for word in sample_words:
    corrupted = corrupt_word(word)
    corrected = correct_word(corrupted)
    results.append((word, corrupted, corrected))

In [7]:
char_total = 0
char_correct = 0

word_total = 0
word_correct = 0

edit_sum = 0

for gold, corrupted, predicted in results:
    for c1, c2 in zip(gold, predicted):
        char_total += 1
        if c1 == c2:
            char_correct += 1

    word_total += 1
    if gold == predicted:
        word_correct += 1

    edit_sum += edit_distance(gold, predicted)

char_acc = char_correct / max(char_total, 1)
word_acc = word_correct / word_total
edit_avg = edit_sum / word_total

print('Character accuracy:', char_acc)
print('Word accuracy:', word_acc)
print('Average edit distance:', edit_avg)

Character accuracy: 0.977409276616198
Word accuracy: 0.92
Average edit distance: 0.127


In [8]:
correct_cnt = 0
print('Correctly fixed words:\n')

for real_world, corrupted, predicted in results:
    if corrupted != real_world and predicted == real_world:
        correct_cnt += 1
        print(corrupted, "->", predicted, "| real_world:", real_world)

print()
print('Fixed a total of', correct_cnt, 'words')

Correctly fixed words:

ეედავება -> ედავება | real_world: ედავება
გააფთღებით -> გააფთრებით | real_world: გააფთრებით
შშეუხრია -> შეუხრია | real_world: შეუხრია
ეხებდოა -> ეხებოდა | real_world: ეხებოდა
იჭრთ -> ჭირთ | real_world: ჭირთ
ელიოტკ -> ელიოტი | real_world: ელიოტი
მოსამშახურეთზ -> მოსამსახურეთა | real_world: მოსამსახურეთა
ტვალიანი -> თვალიანი | real_world: თვალიანი
ფხიძლად -> ფხიზლად | real_world: ფხიზლად
წყალთაშა -> წყალთასა | real_world: წყალთასა
ზღარტანი -> ზღართანი | real_world: ზღართანი
განმსაძღვრელ -> განმსაზღვრელ | real_world: განმსაზღვრელ
წყოობილებას -> წყობილებას | real_world: წყობილებას
ჩატვლით -> ჩათვლით | real_world: ჩათვლით
ნივთშ -> ნივთს | real_world: ნივთს
სასჯელიშ -> სასჯელის | real_world: სასჯელის
ეშარიგა -> შეარიგა | real_world: შეარიგა
მოუნდესს -> მოუნდეს | real_world: მოუნდეს
გასაზფდელად -> გასაზრდელად | real_world: გასაზრდელად
არათანნმიმდევრულად -> არათანმიმდევრულად | real_world: არათანმიმდევრულად
მოხსენიებიშ -> მოხსენიების | real_world: მოხსენიების
გგაიარეთ ->

In [9]:
incorrect_cnt = 0
print('Not fixed words:\n')

for real_world, corrupted, predicted in results:
    if corrupted != real_world and predicted != real_world:
        incorrect_cnt += 1
        print(corrupted, "->", predicted, "| real_world:", real_world)

print('Couldn\'t fix a total of', incorrect_cnt, 'words')

Not fixed words:

გამოიკვთეოს -> გამოიკვთეოს | real_world: გამოიკვეთოს
ბავყოფთ -> ბავყოფთ | real_world: გავყოფთ
ააღიანი -> არიანი | real_world: აღაიანი
მოენდისკენ -> მოენდისკენ | real_world: მოედნისკენ
შინაარწობრივ -> შინაარწორივ | real_world: შინაარსობრივ
მოღვწაეობამ -> მოღვწარობამ | real_world: მოღვაწეობამ
საწთუმალი -> საწთუმალი | real_world: სასთუმალი
გლეეხის -> გელესი | real_world: გლეხის
აღვნშინავთ -> აღვნშინავთ | real_world: აღვნიშნავთ
დანბელების -> დანბელების | real_world: დაბნელების
ნაბიჭვრაო -> ნაბიჭვრაო | real_world: ნაბიჭვარო
ემასლაატებოდა -> ემასლატებოდა | real_world: ემასლაათებოდა
ბოსნიუღი -> ბოშინური | real_world: ბოსნიური
სქვალდებულოა -> სქვალდებულოა | real_world: სავალდებულოა
აკრედტიირებული -> აკრედტირებული | real_world: აკრედიტირებული
სააზუღგეს -> სააურგეს | real_world: საზურგეს
ორლი -> ორლი | real_world: როლი
შტოკ -> შტოკ | real_world: სტოკ
ახლომხდეველი -> ახლომხდეველი | real_world: ახლომხედველი
გაგრძწლდა -> გაგრძწლდა | real_world: გაგრძელდა
თოდირა -> თოდირა | real_wo